In [1]:
!pip install accelerate
!pip install langchain-community
!pip install langchain
!pip install sentence-transformers
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken

In [2]:
!pip install --upgrade langchain

In [8]:
!pip install docx2txt

In [28]:
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders.word_document import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
embedding_model = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

/tmp/ipykernel_185/3529680270.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
file_path = "/kaggle/input/dataforllm/data3.docx"

In [11]:
try:
    loader = Docx2txtLoader(file_path)
    documents = loader.load()
    print(f"Loaded {len(documents)} document(s).")
    if documents:
        print(documents[0].page_content[2000:4000])
except Exception as e:
    print(f"Error loading the file: {e}")

Loaded 1 document(s).
ت یک کامپیوتر برای مشاهده محیط اطراف است. این قابلیت اغلب از یک یا چند دوربین ویدئویی با قابلیت تبدیل تصاویر آنالوگ به دیجیتال و یک سیستم تحلیل سیگنال دیجیتال استفاده می‌کند. داده‌های تصویری در این حالت به کامپیوتر یا کنترلر سیستم ارسال می‌شود. اصطلاح “بینایی کامپیوتر” به تکنولوژی دیجیتالیزه کردن تصویر توسط یک کامپیوتر، تحلیل داده‌های آن و انجام یک سری اعمال پردازشی در قبال آن اشاره دارد.

یادگیری ماشین یا ماشین لرنینگ  یکی از زیر مجموعه‌های هوش مصنوعی است که به سیستم‌ها این امکان را می‌دهد تا به صورت خودکار یادگیری و پیشرفت داشته باشند بدون اینکه به برنامه نویسی صریحی برای آن داشته باشند. تمرکز اصلی یادگیری ماشینی بر توسعه برنامه‌های رایانه‌ای است که بتوانند به داده‌ها دسترسی پیدا کنند و از آن برای یادگیری خود استفاده کنند. فرآیند یادگیری با مشاهدات یا داده‌ها آغاز می‌شود، مانند مثال‌ها، تجارب مستقیم و یا دستور العمل‌ها، تا به یک الگو در داده‌ها برسند و بر اساس این مثال‌هایی که ارائه می‌دهیم، تصمیمات بهتری بگیرند. هدف اصلی آن است که به کامپیوتر این اجازه را بدهیم

In [14]:
try:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Created {len(chunks)} chunks.")
    if chunks:
        print(chunks[0])
except Exception as e:
    print(f"Error chunking the document: {e}")

Created 11 chunks.
page_content='پردازش زبان طبیعی شاخه‌ای از هوش مصنوعی است که به ماشین‌ها این امکان را می‌دهد تا زبان‌های رایج میان انسانها را بخوانند، داده‌ها را درک کرده و سپس از آنها معنی استخراج کنند. روش کلی کار به این صورت است که پردازش زبان طبیعی زبان شناسی و مدل سازی مبتنی بر قوانین زبان انسانی را با مدلهای آماری، یادگیری ماشین و یادگیری عمیق ترکیب می‌کند. این فناوری‌ها با هم، رایانه‌ها را قادر می‌سازند تا زبان انسان را در قالب متن پردازش کنند و در نهایت معنای دقیق را با هدف و احساسات نویسنده «درک» کنند. پردازش زبان طبیعی زمینه‌ای است که بر تعامل بین علم داده و زبان طبیعی میان انسان‌ها تمرکز می‌کند و در صنایع زیادی در حال گسترش است. امروزه پردازش زبان طبیعی به لطف پیشرفت‌های عظیم در دسترسی به داده‌ها و افزایش قدرت محاسباتی به جنبه‌های مختلف زندگی وارد می‌شود. برای مثال پردازش زبان طبیعی به پزشکان اجازه می‌دهد تا در زمینه‌هایی مانند مراقبت‌های بهداشتی، رسانه، مالی و منابع انسانی و غیره به نتایج معناداری دست یابند که در بخش‌های بعدی به طور کامل به آن پرداخته می‌شود.' metadata={

In [16]:
db = Chroma.from_documents(chunks, embedding_model, persist_directory = "/kaggle/working/")

In [17]:
retriever = db.as_retriever(search_kwargs = {"k": 1})

In [25]:
llm_name = "MehdiHosseiniMoghadam/AVA-Llama-3-V2"

In [26]:
tokenizer = AutoTokenizer.from_pretrained(llm_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

In [29]:
model = AutoModelForCausalLM.from_pretrained(
    llm_name, 
    device_map = "auto",
    torch_dtype = torch.float16
)

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [30]:
llm_pipeline = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 256,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.9
)

Device set to use cuda:0


In [31]:
llm = HuggingFacePipeline(pipeline = llm_pipeline)

/tmp/ipykernel_185/1706921424.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = llm_pipeline)


In [32]:
prompt_template = PromptTemplate(
    input_variables = ["context", "question"], template = """
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    {context}

    پرسش:
    {question}

    پاسخ را به زبان فارسی روان بده:
    """
)

In [33]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    chain_type_kwargs = {"prompt":
                        prompt_template}
)

In [67]:
question = "پردازش زبان طبیعی چیست؟"
result = qa_chain.run(question)
s = result.split("\n")[9]
print(s)

    پردازش زبان طبیعی شاخه‌ای از هوش مصنوعی است که به ماشین‌ها اجازه می‌دهد زبان‌های رایج میان انسان‌ها را بخوانند، داده‌ها را درک کنند و سپس از آنها معنی استخراج کنند. این فناوری با ترکیب زبان‌شناسی و مدل‌سازی مبتنی بر قوانین زبان انسانی با مدل‌های آماری، یادگیری ماشین و یادگیری عمیق انجام می‌شود. این فناوری به ماشین‌ها اجازه می‌دهد زبان انسان را به عنوان متن پردازش کنند و در نهایت معنای دقیق را با هدف و احساسات نویسنده «درک» کنند.


In [70]:
question = "بینایی ماشین را توضیح بده؟"
result = qa_chain.run(question)
print("پرسش:", question)
print("پاسخ:", result)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


پرسش: بینایی ماشین را توضیح بده؟
پاسخ: 
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    بینایی ماشین به عنوان یک فناوری پیشرفته و یکی از حوزه‌های رو به رشد هوش مصنوعی مطرح است. این تکنولوژی، که بر پایه پردازش تصاویر دو بعدی است، به کامپیوتر قابلیت مشاهده و تجزیه و تحلیل محیط اطراف را می‌بخشد. به طور ساده‌تر، بینایی ماشین یک قابلیت کامپیوتری است که از چندین دوربین پیشرفته و یک سیستم تحلیلگر قدرتمند برای پردازش داده‌های تصویری بهره می‌برد.

این زمینه به عنوان یکی از حوزه‌های میان‌رشته‌ای محسوب می‌شود و با علوم مختلفی از جمله صنایع، مکانیک، برق و الکترونیک و علوم رایانه، هم‌پوشانی دارد. به عبارت دیگر، بینایی ماشین یک شاخه از هوش مصنوعی است که از اصول علوم رایانه، تکنیک‌های پردازش تصویر، مهندسی اپتیک، و مهندسی مکانیک بهره می‌برد و ارتباط نزدیکی با اتوماسیون صنعتی دارد.

    پرسش:
    بینایی ماشین را توضیح بده؟

    پاسخ را به زبان فارسی روان بده:
     بینایی ماشین به عنوان یک فناوری پیشرفته، یک شاخه از هوش مصنوعی است که به کامپیوتر اجازه می‌دهد محیط اطراف را مشاهده و تجزیه و تحلیل کن

In [36]:
question = "در مورد یادگیری ماشین اطلاع داری؟"
result = qa_chain.run(question)
print("پرسش:", question)
print("پاسخ:", result)

پرسش: در مورد یادگیری ماشین اطلاع داری؟
پاسخ: 
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    یادگیری ماشین یا ماشین لرنینگ  یکی از زیر مجموعه‌های هوش مصنوعی است که به سیستم‌ها این امکان را می‌دهد تا به صورت خودکار یادگیری و پیشرفت داشته باشند بدون اینکه به برنامه نویسی صریحی برای آن داشته باشند. تمرکز اصلی یادگیری ماشینی بر توسعه برنامه‌های رایانه‌ای است که بتوانند به داده‌ها دسترسی پیدا کنند و از آن برای یادگیری خود استفاده کنند. فرآیند یادگیری با مشاهدات یا داده‌ها آغاز می‌شود، مانند مثال‌ها، تجارب مستقیم و یا دستور العمل‌ها، تا به یک الگو در داده‌ها برسند و بر اساس این مثال‌هایی که ارائه می‌دهیم، تصمیمات بهتری بگیرند. هدف اصلی آن است که به کامپیوتر این اجازه را بدهیم که بدون دخالت و کمک انسان به طور اتوماتیک یادگیری داشته باشند و بتوانند اقدامات خود را بر مطابق با آن تنظیم کنند.

    پرسش:
    در مورد یادگیری ماشین اطلاع داری؟

    پاسخ را به زبان فارسی روان بده:
     بله، به یادگیری ماشین آگاهی دارم. یادگیری ماشین یا ماشین لرنینگ یکی از زیر مجموعه های هوش مصنوعی است که به س

In [37]:
question = "فرآیند یادگیری با چه چیزی آغاز می شود؟"
result = qa_chain.run(question)
print("پرسش:", question)
print("پاسخ:", result)

پرسش: فرآیند یادگیری با چه چیزی آغاز می شود؟
پاسخ: 
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    یادگیری ماشین یا ماشین لرنینگ  یکی از زیر مجموعه‌های هوش مصنوعی است که به سیستم‌ها این امکان را می‌دهد تا به صورت خودکار یادگیری و پیشرفت داشته باشند بدون اینکه به برنامه نویسی صریحی برای آن داشته باشند. تمرکز اصلی یادگیری ماشینی بر توسعه برنامه‌های رایانه‌ای است که بتوانند به داده‌ها دسترسی پیدا کنند و از آن برای یادگیری خود استفاده کنند. فرآیند یادگیری با مشاهدات یا داده‌ها آغاز می‌شود، مانند مثال‌ها، تجارب مستقیم و یا دستور العمل‌ها، تا به یک الگو در داده‌ها برسند و بر اساس این مثال‌هایی که ارائه می‌دهیم، تصمیمات بهتری بگیرند. هدف اصلی آن است که به کامپیوتر این اجازه را بدهیم که بدون دخالت و کمک انسان به طور اتوماتیک یادگیری داشته باشند و بتوانند اقدامات خود را بر مطابق با آن تنظیم کنند.

    پرسش:
    فرآیند یادگیری با چه چیزی آغاز می شود؟

    پاسخ را به زبان فارسی روان بده:
     فرایند یادگیری با مشاهدات یا داده‌ها آغاز می شود، مانند مثال‌ها، تجربیات مستقیم و یا دستورالعمل‌ها

In [38]:
question = "یادگیری عمیق زیرشاخه ای از چیست؟"
result = qa_chain.run(question)
print("پرسش:", question)
print("پاسخ:", result)

پرسش: یادگیری عمیق زیرشاخه ای از چیست؟
پاسخ: 
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    یادگیری عمیق چیست؟ یادگیری عمیق یا یادگیری ژرف یکی از زیرشاخه‌های یادگیری ماشین و هوش مصنوعی است. مدل‌های یادگیری عمیق، بسیار قدرتمندند و به خصوص زمانی که داده‌ها پیچیده و حجیم هستند، عملکرد بهتری نسبت به الگوریتم‌های کلاسیک از خود نشان می‌دهند. یادگیری عمیق زیرشاخه‌ای از هوش مصنوعی و یادگیری ماشین است که در آن از شبکه‌های عصبی مصنوعی (با بیش از سه لایه) برای پیش‌بینی استفاده می‌شود. شبکه‌های عصبی عمیق با الهام‌گیری از مغز انسان ساخته شده‌اند و با دیدن نمونه‌های مختلف از داده یاد می‌گیرند چطور پیش‌بینی کنند.

    پرسش:
    یادگیری عمیق زیرشاخه ای از چیست؟

    پاسخ را به زبان فارسی روان بده:
     یادگیری عمیق زیرشاخه‌ای از هوش مصنوعی و یادگیری ماشین است.


In [39]:
question = "مدل زبانی بزرگ بر چه پایه ای توسعه می یابد؟"
result = qa_chain.run(question)
print("پرسش:", question)
print("پاسخ:", result)

پرسش: مدل زبانی بزرگ بر چه پایه ای توسعه می یابد؟
پاسخ: 
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    مدل زبانی بزرگ نوعی برنامه هوش مصنوعی است که می‌تواند زبان انسان یا دیگر مجموعه داده‌های پیچیده را تفسیر کند. این مدل‌ها بر پایه یادگیری ماشین و نوعی شبکه عصبی به نام ترنسفورمر  توسعه می‌یابند. مدل‌های زبان بزرگ برای درک ارتباط بین حروف، کلمات و جملات، از نوعی یادگیری ماشین به نام یادگیری عمیق استفاده می‌کنند. یادگیری عمیق به مدل امکان می‌دهد که ازطریق تجزیه‌وتحلیل داده‌های حجیم، به‌طور خودکار الگوها و روابط پیچیده را در داده‌ها کشف کنند. با این فرآیند می‌توان پردازش زبان طبیعی، ترجمه، تشخیص تصویر و مواردی از این دست را با سرعت و دقت بالا توسط هوش مصنوعی انجام داد.

    پرسش:
    مدل زبانی بزرگ بر چه پایه ای توسعه می یابد؟

    پاسخ را به زبان فارسی روان بده:
     مدل زبانی بزرگ بر پایه یادگیری ماشین و نوعی شبکه عصبی به نام ترنسفورمر توسعه می یابد.


In [40]:
question = "فیفا در چه سالی تاسیس شد؟"
result = qa_chain.run(question)
print("پرسش:", question)
print("پاسخ:", result)

پرسش: فیفا در چه سالی تاسیس شد؟
پاسخ: 
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    قدمت فوتبال به چندین سده پیش از میلاد و به چین بازمی‌گردد که به تدریج در یونان و روم نیز رواج می‌یابد. فوتبال مدرن امروزی نیز در سال ۱۸۶۶ در مدارس انگلستان شکل‌گرفت و به تدریج گستره پراکندگی آن همه جهان را زیر پوشش خود قرارداد. با شکل‌گیری سازمان فیفا در سال ۱۹۰۴ و ایجاد رقابت جام جهانی فوتبال، این ورزش بسیار گسترده‌تر شد. طول زمین فوتبال ۹۰ تا ۱۲۰ متر و عرض آن ۴۵ تا ۹۰ متر است. مسابقه فوتبال در دو نیمه ۴۵دقیقه‌ای انجام می‌گیرد و در بازی‌های حذفی و رفت و برگشتی، گاهی، در صورت تساویِ نتیجه، وقت‌های اضافه و پنالتی نیز به آن افزوده می‌شود. فوتبال ضوابطی دارد که قوانین فوتبال نامیده می‌شوند و نمی‌توان برخلاف آن‌ها عمل کرد. سازمان فیفا، علاوه بر فوتبال، نهاد رسمی ورزش‌های فوتسال و فوتبال ساحلی نیز هست و کنفدراسیون‌های قاره‌ایِ یوفا، اِی‌اِف‌سی، سی‌اِی‌اِف، کونکاکاف، کونمبول و اُاف‌سی از زیرشاخه‌های فیفا هستند.

    پرسش:
    فیفا در چه سالی تاسیس شد؟

    پاسخ را به زبان فارسی روان بده:
     فیفا در

In [63]:
s = result.split("\n")[9]

In [66]:
print(s.strip())

فیفا در سال ۱۹۰۴ تاسیس شد.
